In [2]:
import snscrape.modules.telegram as telegram

scraper = telegram.TelegramChannelScraper('fatpigsignals')

for i,tel in enumerate(scraper.get_items()):
  if i >50:
    break
  print(f"{i} content: {tel.content} date : {tel.date}")

0 content: Binance, Bittrex, KuCoin, FTX, Huobi.pro#ETH/BTC Take-Profit target 2 ✅Profit: 8.33% 📈Period: 48 Minutes ⏰ date : 2022-07-16 16:45:26+00:00
1 content: ETH PoS merger expected September 19th 👀 https://cointelegraph.com/news/ethereum-dev-confirms-perpetual-date-for-pos-merge date : 2022-07-16 16:45:25+00:00
2 content: Bittrex, FTX#UNI/BTC Take-Profit target 5 ✅Profit: 41.10% 📈Period: 18 Days 17 Hours 23 Minutes ⏰ date : 2022-07-09 04:34:53+00:00
3 content: Binance#DYDX/BTC Take-Profit target 4 ✅Profit: 24.37% 📈Period: 10 Days 14 Hours 55 Minutes ⏰ date : 2022-07-04 22:12:16+00:00
4 content: 👆   update on our previous projection for bitcoin on the weekly chart .   As shown in the chart , prices have been going down relentlessly towards our target of ~15k .   This represents a 71% drawdown from  its peak and a 68% drawdown from our red ball in… date : 2022-07-02 13:47:11+00:00
5 content: Binance#EGLD/BTC Take-Profit target 1 ✅Profit: 4.17% 📈Period: 4 Hours 39 Minutes ⏰ date : 20